In [1]:
import os
from Bio.PDB import PDBParser, PDBIO, Select, Atom

# Directory containing PDB files
pdb_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"
output_dir = "/home/hp/nayanika/github/GPX6/prep_structures/"
os.makedirs(output_dir, exist_ok=True)

# List of PDB files
pdb_files = ["original_humansec.pdb"]

# List of mutations (residue_number, new_residue)
mutations = {
    47: "SER",  # Mutate residue at position 47 to Serine
}

class MutateSelect(Select):
    def __init__(self, mutations):
        self.mutations = mutations

    def accept_residue(self, residue):
        res_id = residue.get_id()[1]
        if res_id in self.mutations:
            new_residue = self.mutations[res_id]
            residue.resname = new_residue
            
            # If mutating to SER, ensure the OG atom is present
            if new_residue == "SER":
                self.ensure_ser_og(residue)
                
        return True

    def ensure_ser_og(self, residue):
        """Ensure OG atom is present in Serine (SER) residue."""
        atom_names = [atom.get_name() for atom in residue]
        if "OG" not in atom_names:
            cb = residue['CB']
            # Manually adding OG atom coordinates based on CB atom
            og_coords = [cb.coord[0] - 1.341, cb.coord[1] + 0.33, cb.coord[2] + 0.132]  # Offset values to simulate OG position
            og_atom = Atom.Atom(
                "OG", og_coords, 1.0, 1.0, " ", "OG", residue.get_id()[1], element="O"
            )
            residue.add(og_atom)
            print(f"Added OG atom to SER at residue {residue.get_id()[1]} with coords {og_coords}")

# Function to mutate residues in a PDB file
def mutate_pdb(pdb_file, output_file, mutations):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_file)
    
    io = PDBIO()
    io.set_structure(structure)
    
    # Save the mutated structure without removing any atoms
    io.save(output_file, MutateSelect(mutations))
    print(f"Mutated PDB saved to {output_file}")

# Apply mutations to all PDB files
for pdb_file in pdb_files:
    pdb_path = os.path.join(pdb_dir, pdb_file)
    output_path = os.path.join(output_dir, "A47S.pdb")
    mutate_pdb(pdb_path, output_path, mutations)

print("All mutations completed successfully.")


Added OG atom to SER at residue 47 with coords [36.658000549316405, 30.986999588012694, 18.88800051879883]
Mutated PDB saved to /home/hp/nayanika/github/GPX6/prep_structures/A47S.pdb
All mutations completed successfully.
